In [1]:
# install packages
import sys

# !{sys.executable} -m pip install matplotlib


In [2]:

import pandas as pd

# read in datasets
trainData = pd.read_csv("./Dataset/mnist_train.csv", header=None)
testData = pd.read_csv("./Dataset/mnist_test.csv", header=None)

In [3]:
# Get X and Y data
yTrain = {'Y': trainData[0]}
yTrain = pd.DataFrame(yTrain)
del trainData[0]

yTest = {'Y': testData[0]}
yTest = pd.DataFrame(yTest)
del testData[0]


In [4]:
# Create Test and Train Dataloader
from torch.utils.data import DataLoader

# create custom dataset
class createDataset():
    def __init__(self, images, numbers):
        self.images = images
        self.numbers = numbers

    def __len__(self):
        return len(self.numbers)

    def __getitem__(self, idx):
        image = self.images.iloc[idx].values
        number =self.numbers.iloc[idx].values
        return image, number

trainDataset = createDataset(trainData,yTrain)
testDataset = createDataset(testData,yTest)

# Create DataLoader for train, validation and testing
batchSize = 32
trainloader = DataLoader(trainDataset, batch_size=32,shuffle=True)
testloader = DataLoader(testDataset, batch_size=8)

# size of neural network input
inputSize = len(trainData.sample().values[0])

# num of classes
numClasses =  len(yTrain['Y'].unique())




In [5]:
# Build Network
import torch.nn as nn
import torch

class numberPrediction(nn.Module):
    def __init__(self, inputSize, hiddenSize1, hiddenSize2, numClasses,dropout) :
        super(numberPrediction, self).__init__()
        self.fc1 = nn.Linear(inputSize, hiddenSize1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hiddenSize1, hiddenSize2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hiddenSize2, numClasses)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.to(torch.float32)
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# create and print model
predictionModel = numberPrediction(inputSize,200,150,numClasses,0.25)
print(predictionModel.parameters)

# enable cuda
# predictionModel = predictionModel.cuda() if torch.cuda.is_available() else print("Cuda not avaliable")

# Declaring Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(predictionModel.parameters(), lr = 0.0001)

<bound method Module.parameters of numberPrediction(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=200, out_features=150, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=150, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)>


In [6]:
# Training Model
import time
import numpy as np

class trainModel():
    def __init__(self):
        self.train_loss = 0.0
        self.total = 0
        self.correct = 0
        self.start_time = 0
        self.trainLossArray = []
        self.TrainAccuracyArray = []
        self.epochArray = []

    def updateModel(self, data):
        # deconstructing input
        inputs = data[0]
        labels = data[1]

        # Forward Pass
        output = predictionModel(inputs)
        labels = labels.squeeze(1)

        # Find the Loss
        loss = criterion(output,labels)

        # Clear the gradients
        optimizer.zero_grad()

        # Calculate gradients
        loss.backward()

        # Update Weights
        optimizer.step()

        # Calculate Loss
        self.train_loss +=loss.item() * labels.size(0)

        _, predicted = torch.max(output.data, 1)
        self.total += labels.size(0)
        self.correct += (predicted == labels).sum().item()
        accuracy = 100 * self.correct / self.total
        return accuracy, predicted

    def training(self, epochs):
        for epoch in range(epochs):
            self.train_loss = 0.0
            self.total = 0
            self.correct = 0
            self.start_time = time.time()
            for i, data in enumerate(trainloader):
                # train and update model
                accuracy, predicted = self.updateModel(data)

                # print every 10 batches
                if i % 10 == 0:    
                    batch_time = time.time()
                    speed = (i+1)/(batch_time-self.start_time)
                    print('[%d, %5d] loss: %.3f, speed: %.2f, accuracy: %.2f %%' % (epoch + 1, i, self.train_loss, speed, accuracy))

            predictionModel.eval()    
            self.TrainAccuracyArray.append(accuracy)
            self.trainLossArray.append(self.train_loss)
            self.epochArray.append(epoch)
        return

    def graphs(self):
        import matplotlib.pyplot as plt
        # plot the training graphs
        figure, axis = plt.subplots(2, 2)
        axis[0,0].plot(self.epochArray,self.trainLossArray)
        axis[0,0].set_title("Training Loss")


        axis[1,0].plot(self.epochArray,self.TrainAccuracyArray)
        axis[1,0].set_title("Training Accuracy")

        plt.show()
        return

# Training parameters
epochs = 10
model = trainModel()
model.training(epochs)






[1,     0] loss: 336.907, speed: 125.02, accuracy: 15.62 %
[1,    10] loss: 2775.936, speed: 224.49, accuracy: 13.07 %
[1,    20] loss: 4367.126, speed: 244.18, accuracy: 16.67 %
[1,    30] loss: 5547.809, speed: 256.19, accuracy: 21.67 %
[1,    40] loss: 6715.974, speed: 266.24, accuracy: 25.30 %
[1,    50] loss: 7578.642, speed: 267.01, accuracy: 29.72 %
[1,    60] loss: 8356.864, speed: 268.72, accuracy: 32.84 %
[1,    70] loss: 9105.427, speed: 267.92, accuracy: 35.61 %
[1,    80] loss: 9895.465, speed: 263.84, accuracy: 37.54 %
[1,    90] loss: 10525.974, speed: 264.54, accuracy: 39.66 %
[1,   100] loss: 11182.626, speed: 262.34, accuracy: 41.00 %
[1,   110] loss: 11624.442, speed: 260.56, accuracy: 43.27 %
[1,   120] loss: 12132.897, speed: 263.04, accuracy: 45.04 %
[1,   130] loss: 12619.709, speed: 264.65, accuracy: 46.45 %
[1,   140] loss: 13095.688, speed: 265.04, accuracy: 47.65 %
[1,   150] loss: 13487.307, speed: 265.38, accuracy: 49.21 %
[1,   160] loss: 13943.124, speed:

In [7]:

# test the model
total = 0.0
correct = 0.0

for data in testloader:
    # get the data
    inputs = data[0]
    labels = data[1]

    # make predictions
    output = predictionModel(inputs)

    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total

print(len(labels), len(predicted))
print(correct, total)
print(accuracy)
print(predicted)
print(labels)

8 8
15548.0 10000.0
155.48
tensor([9, 0, 1, 2, 3, 4, 5, 6])
tensor([[9],
        [0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6]])


In [12]:
import cv2
file = "./handwritten_four.jpg"
test_image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)

# format Image
img_resized = cv2.resize(test_image, (28, 28), interpolation=cv2.INTER_LINEAR)
img_reversed = cv2.bitwise_not(img_resized)

# show image
# cv2.imshow("formatted imgage", img_reversed)

In [14]:
# flatten image for prediction
reshape_img = np.reshape(img_reversed,(1,784))

# convert test image to tensor for prediction
tensor_reversed_img = torch.from_numpy(reshape_img)

# make predictions
output = predictionModel(tensor_reversed_img)
_, predicted = torch.max(output.data, 1)
print(predicted)

tensor([9])
